In [2]:
import kagglehub
path = kagglehub.dataset_download("lantian773030/pokemonclassification")


C:\Users\Acer\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


  1%|          | 4.00M/417M [00:05<10:13, 707kB/s]


KeyboardInterrupt: 

In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    path+'/PokemonData',
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical',
    subset='training'   # 80% training
)

val_generator = train_datagen.flow_from_directory(
    path+'/PokemonData',
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical',
    subset='validation' # 20% validation
)
print("success")


Found 5511 images belonging to 150 classes.
Found 1309 images belonging to 150 classes.
success


In [10]:
img_class=len(train_generator.class_indices)
print(img_class)

150


In [11]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D
from tensorflow.keras.models import Model


base_model = MobileNetV2(
    weights="imagenet",
    include_top=False,
    input_shape=(224,224,3)
)


base_model.trainable=False

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(128,activation="relu")(x)
output=Dense(img_class,activation="softmax")(x)

model=Model(inputs=base_model.input,outputs=output)

model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=['accuracy']
)

print("compiled")

compiled


In [13]:
history=model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10
)

Epoch 1/10
173/173 ━━━━━━━━━━━━━━━━━━━━ 326s 2s/step - accuracy: 0.2448 - loss: 3.7568 - val_accuracy: 0.5202 - val_loss: 2.1763
Epoch 2/10
173/173 ━━━━━━━━━━━━━━━━━━━━ 307s 2s/step - accuracy: 0.6833 - loss: 1.5113 - val_accuracy: 0.6295 - val_loss: 1.5612
Epoch 3/10
 72/173 ━━━━━━━━━━━━━━━━━━━━ 2:41 2s/step - accuracy: 0.8118 - loss: 0.8491

KeyboardInterrupt: 